In [ ]:
import pandas as pd
import numpy as np
!pip install st_dbscan
from st_dbscan import ST_DBSCAN
from datetime import datetime
import time
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import pandas as pd
from sklearn.cluster import DBSCAN
import seaborn as sns
from itertools import product
from sklearn.preprocessing import MinMaxScaler
import matplotlib.colors as mcolors
import sys
sys.path.insert(0, "../src")
from ST_Cluster import *

# Clustering Params

In [ ]:
space_thres_list = [0.5, 1, 2, 3, 4]
time_thres_list = [1, 3, 5, 7, 14, 30]
num_thres_list = [3, 5, 7, 10]

# Create a folder for temporary files
temp_folder = "Clusters"
os.makedirs(temp_folder, exist_ok=True)

# Disaster Data Load

In [ ]:
disasters = pd.read_csv("../../final_filtered_disasters.csv")

# Ensure that 'fullFIPS' is properly formatted as a 5-character string
disasters['fullFIPS'] = disasters['fullFIPS'].astype(int).astype(str)
disasters['fullFIPS'] = disasters['fullFIPS'].apply(lambda x: str(x).zfill(5))

# Preprocess 'incidentBeginDate' to numeric days
disasters['incidentBeginDate'] = pd.to_datetime(disasters['incidentBeginDate'])  # Ensure column is datetime type

origin_date = disasters['incidentBeginDate'].min()
disasters['daysSinceStart'] = (disasters['incidentBeginDate'] - origin_date).dt.days
disasters['date'] = pd.to_datetime(disasters['daysSinceStart'], unit='D', origin=origin_date)

# Create a unique index for each row
disasters['index'] = disasters.index

In [ ]:
# Path to your shapefile
county_shapefile_path = '../../../Local_Data/Geospatial/tl_2019_us_county.shp'

# Load the shapefile using GeoPandas
gdf = gpd.read_file(county_shapefile_path)

# Ensure the GEOID column exists (modify if your column has a different name)
if 'GEOID' not in gdf.columns:
    raise ValueError("The GEOID column is not found in the shapefile.")

# Calculate centroids and extract coordinates to two decimal places
gdf['centroid_lat'] = gdf.geometry.centroid.y.round(2)
gdf['centroid_lon'] = gdf.geometry.centroid.x.round(2)

# Create a DataFrame with GEOID, latitude, and longitude
centroid_df = gdf[['GEOID', 'centroid_lat', 'centroid_lon']]

# Ensure that 'fullFIPS' is properly formatted as a 5-character string
centroid_df['GEOID'] = centroid_df['GEOID'].astype(int).astype(str)
centroid_df['GEOID'] = centroid_df['GEOID'].apply(lambda x: str(x).zfill(5))

# Save to CSV
output_csv_path = 'county_centroids.csv'
centroid_df.to_csv(output_csv_path, index=False)

print(f"Centroid data saved to {output_csv_path}")

In [ ]:
# Merge the latitude and longitude columns onto the disasters dataframe
disasters = disasters.merge(
    centroid_df[['GEOID', 'centroid_lat', 'centroid_lon']],
    left_on='fullFIPS',
    right_on='GEOID',
    how='left'
)

# Drop the GEOID column if you don't need it
disasters = disasters.drop(columns=['GEOID'])

disaster_sensitivity(disasters, space_thres_list, time_thres_list, num_thres_list)
# Save only the index and cluster columns
disasters.to_csv(f"{temp_folder}/clustered_disasters_sensitivity.csv", index=False)

In [ ]:
# Save the result
disasters.to_csv(f"{temp_folder}/clustered_disasters.csv", index=False)

print("Clustering complete. Output saved to 'clustered_disasters.csv'.")